# Job Scheduling Problem

## Import packages and data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as stat
import gurobipy as gp
from gurobipy import GRB
from tqdm import tqdm
from collections import defaultdict
import random

In [2]:
df_garages = pd.read_excel(r"C:\Users\dangt\OneDrive\Documents\JOBS\Applications\KTP\Portsmouth - Routing and Scheduling Optimisation\RSO Data Set.xlsx", sheet_name='Garages (Static Data)', header = 4, nrows=6)
df_garages

,Garage ID,Garage Name,Location X,Location Y,Level 1,Level 2,Level 3,Truck 1,Truck 2
0,G001,ABL 1 A,100,150,1,1,2,1,1
1,G002,ABL 1 B,250,80,0,1,2,0,1
2,G003,ABL 1 C,380,200,0,0,2,1,0
3,G004,ABL 1 D,50,280,1,1,1,1,0
4,G005,ABL 1 E,220,350,0,1,1,1,1


In [3]:
df_job_types = pd.read_excel(r"C:\Users\dangt\OneDrive\Documents\JOBS\Applications\KTP\Portsmouth - Routing and Scheduling Optimisation\RSO Data Set.xlsx", sheet_name='Job Types (Static Data)', header = 4)
df_job_types

,Job Type ID,Job Type Name,Description,Process Time (hours),Skill Required
0,1,Minor Scratch or Paint Repair,"Sanding, priming, colour-matching, painting, a...",4,Level 1
1,2,Bumper Replacement or Repair,"Removing damaged bumper, fitting new one, pain...",5,Level 2
2,3,Panel Beating / Dent Repair,"Straightening dents, applying filler, sanding,...",3,Level 2
3,4,Door Replacement / Realignment,"Replacing or aligning door structure, wiring, ...",5,Level 2
4,5,Windscreen Replacement,"Removing old glass, cleaning frame, bonding ne...",2,Level 1
5,6,Headlight / Taillight Replacement,Replacing cracked or non-functional lights and...,1,Level 1
6,7,Suspension / Wheel Alignment,Replacing or adjusting suspension components a...,4,Level 3
7,8,Engine or Transmission Repair (Major),"Mechanical teardown, diagnostics, and reassembly.",12,Level 3


In [4]:
df_jobs = pd.read_excel(r"C:\Users\dangt\OneDrive\Documents\JOBS\Applications\KTP\Portsmouth - Routing and Scheduling Optimisation\RSO Data Set.xlsx", sheet_name='Arrival customers (dynamic)', header = 3)
df_jobs.head(5)

,Job ID ↑,Date,Time,Location X,Location Y,Job Type,Part Type Required,Estimated Authrisation Time (days),L_parts
0,JOB-0001,2025-01-01,08:00:00,187.54,285.45,2,2,3.0,3.0
1,JOB-0002,2025-01-01,12:19:20,346.99,131.30,1,3,2.0,0.5
2,JOB-0003,2025-01-01,13:32:20,241.45,39.01,6,1,1.0,2.0
3,JOB-0004,2025-01-01,16:55:43,69.26,116.42,7,1,0.5,2.0
4,JOB-0005,2025-01-02,11:13:22,107.63,230.66,8,3,2.0,0.5


In [5]:
type(df_jobs['Date'].tolist()[0])
# Need to parse dates

str

In [6]:
df_confirmed_jobs = pd.read_excel(r"C:\Users\dangt\OneDrive\Documents\JOBS\Applications\KTP\Portsmouth - Routing and Scheduling Optimisation\RSO Data Set.xlsx", sheet_name='Garage (Dynamic Data)', header = 4)
df_confirmed_jobs = df_confirmed_jobs.iloc[:,:-1]
df_confirmed_jobs.head(5)

,Job ID,Garage,Job Type,Process Time (hours),Skill Required,Joy Status,ETA Parts
0,CJOB-0001,ABL 1 A,1,4,Level 1,S1,0.0
1,CJOB-0002,ABL 1 A,3,3,Level 2,S1,0.0
2,CJOB-0003,ABL 1 A,5,2,Level 1,S2,0.0
3,CJOB-0004,ABL 1 A,3,3,Level 2,S2,0.0
4,CJOB-0005,ABL 1 A,2,5,Level 2,S3,0.0


## Index sets

In [7]:
garages = df_garages['Garage Name'].unique()
G = range(len(garages))
print(f"There are {len(garages)} garages")

There are 5 garages


In [8]:
# Get a list of active jobs (confirmed jobs which need scheduling)
active_jobs = df_confirmed_jobs[df_confirmed_jobs['Joy Status'] == 'S5'] ['Job ID'].values.tolist()
J = range(len(active_jobs))
print(f"There are {len(active_jobs)} active jobs")

There are 50 active jobs


In [9]:
active_jobs[0]

'CJOB-0041'

In [10]:
D = range(365) # Days of the year
print(f"Garages work {len(D)} days in the year")

Garages work 365 days in the year


In [11]:
S = range(1,4)
print(f"There are {len(S)} skill levels")

There are 3 skill levels


## Parameters

In [12]:
# Capacity of each garage
hours_per_staff = 8
Cap = {} # Dictionary to store capacities
for garage in garages:
    Cap[garage] = {} # Dictionary to store each garage's capacity by level
    for s in S:
        Cap[garage][s] = df_garages[df_garages['Garage Name'] == garage][f'Level {s}'].values[0]

Cap['ABL 1 A']

{1: np.int64(1), 2: np.int64(1), 3: np.int64(2)}

In [13]:
# Flexible start window for each job
D_soonest_start = {} # Dictionary to store soonest start dates by job
for job in active_jobs:
    # Get the expected parts lead time
    D_eta = df_confirmed_jobs[df_confirmed_jobs['Job ID'] == job]['ETA Parts'].values[0]

    # Randomise the expected garage availability date
    # random.seed(23)
    D_garage = random.randint(0, 10)
    
    D_soonest_start[job] = max(D_eta, D_garage)

In [14]:
D_soonest_start['CJOB-0041']

np.float64(3.0)

In [15]:
# Labour requirement
L = {} # Dictionary to store labour requirements
for index, row in df_confirmed_jobs.iterrows():
  cjob = row['Job ID']
  L[cjob] = {} # Dictionary to store each job's labour requirement by level
  for s in S:
    if row['Skill Required'] == f'Level {s}':
      L[cjob][s] = row['Process Time (hours)']
    else:
      L[cjob][s] = 0
L['CJOB-0002']

{1: 0, 2: 3, 3: 0}

In [16]:
# Staff cost
staff_cost = {1: 10, 2: 20, 3: 30}

In [17]:
# # WIP at each garage at the snapshot (day = 0)
# WIP = {}
# for garage in garages:
#   WIP[garage] = pd.DataFrame(0, index = D, columns = [f'Level {level}' for level in S])

#   # Get ongoing jobs in this garage that are ready for repair (S1 and S2)
#   garage_cjobs = df_confirmed_jobs[(df_confirmed_jobs['Garage'] == garage)
#                                    & ((df_confirmed_jobs['Joy Status'] == 'S1') |
#                                       (df_confirmed_jobs['Joy Status'] == 'S2'))]
#   for i, row in garage_cjobs.iterrows():
#     for level in S:
#       if row['Skill Required'] == f'Level {level}':
#         WIP[garage].loc[0, f'Level {level}'] += row['Process Time (hours)']

# WIP['ABL 1 C']


## Model function

In [18]:
def scheduling_model(active_jobs, garages, staff_cost, D_soonest_start):
  """
  """
  model = gp.Model()
  # model.Params.OutputFlag = 0 # Suppress log output

  # Variables
  x = model.addVars(J, G, D, vtype = gp.GRB.BINARY, name="x") # Assignment of a job to a garage
  y = model.addVars(G, D, S, vtype=gp.GRB.CONTINUOUS, name = "y", lb = 0.0) # Idle costs
  t = model.addVars(J, vtype=gp.GRB.CONTINUOUS, name = "t", lb = 0.0) # Actual start date
  h = model.addVars(J, vtype=gp.GRB.CONTINUOUS, name = "h", lb = 0.0) # Lateness (= actual start - last day of flexible window)

  # Constraints
  ## Each job can only be assigned to 1 garage on 1 day
  for j in J:
    model.addConstr(gp.quicksum(x[j, g, d] for g in G for d in D) == 1,
                    name = f"Job {j} assigned to 1 garage")

  ## Job must not exceed the garage capacity at any skill level
  for d in D:
    for g in G:
      for j in J:
        for s in S:
          job_id = active_jobs[j]
          garage = garages[g]
          model.addConstr(x[j, g, d]*L[job_id][s] <= Cap[garage][s],
                          name = f"Job {j} not over garage {g} capacity on day {d}, at skill level {s}")

  ## Accumulated staff requirements of all jobs assigned to a garage must not exceed the garage capacity at any skill level
  for d in D:
    for g in G:
        for s in S:
          garage = garages[g]
          model.addConstr(gp.quicksum(x[j, g, d]*L[active_jobs[j]][s] for j in J) <= Cap[garage][s],
                          f"All jobs not over garage {g} capacity on day {d}, at skill level {s}")

  # Objectives
  ## Minimise idle labour cost
  for d in D:
    for g in G:
      for s in S:
        garage = garages[g]
        idle_hours = Cap[garage][s] - gp.quicksum(x[j, g, d]*L[active_jobs[j]][s] for j in J)
        idle_cost = idle_hours * staff_cost[s]
        model.addConstr(y[g, d, s] == idle_cost, name = f"Idle labour cost in garage {g} on day {d} at skill level {s}")

  ## Minimise variance between promised and actual start/completion time (assume that job durations remain unchanged)
  for j in J:
    model.addConstr(t[j] == gp.quicksum(x[j, g, d] * d for g in G for d in D),
                    name = f"Actual start date of job {j}")
    model.addConstr(h[j] == t[j] - (D_soonest_start[active_jobs[j]] + 2),
                    name = f"Lateness of job {j}")

  # Set objective functions
  obj = y.sum() + h.sum() # Equal weights
  model.setObjective(obj, GRB.MINIMIZE)

  # Model execution
  # model.Params.timeLimit = 10 # Pilot
  model.update()
  model.optimize()

  #Handle infeasibility
  if model.Status == GRB.INFEASIBLE:
      print("Model is infeasible")
      # model.computeIIS()
      # model.write(f"inf_model.ilp")
  else:
      # Print results
      status_meanings = {
                    GRB.OPTIMAL: "OPTIMAL",
                    GRB.UNBOUNDED: "UNBOUNDED",
                    GRB.INF_OR_UNBD: "INFEASIBLE OR UNBOUNDED",
                    GRB.TIME_LIMIT: "TIME LIMIT REACHED",
                    GRB.INTERRUPTED: "INTERRUPTED",
                    GRB.SUBOPTIMAL: "SUBOPTIMAL"
                }
      print(f"Model status: {status_meanings[model.Status]}")
      print(f"Gap: {model.MIPGap:.0%}")
      print(f"Objective (Total underproduction across SKUs): {model.ObjVal:g}")

      # Create a schedule of empty lists
      schedule = pd.DataFrame(index=D, columns=garages)
      for col in schedule.columns:
          schedule[col] = schedule[col].apply(lambda x: [])

      # Fill in the schedule
      for d in D:
        for g in G:
          for j in J:
            if int(round(x[j, g, d].X)) == 1:
              schedule.iloc[d, g].append(active_jobs[j])

      return schedule


In [19]:
test = scheduling_model(active_jobs, garages, staff_cost, D_soonest_start)

Set parameter Username
Set parameter LicenseID to value 2682535
Academic license - for non-commercial use only - expires 2026-06-26
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Optimize a model with 284850 rows, 96825 columns and 461625 nonzeros
Model fingerprint: 0x5cdf199a
Variable types: 5575 continuous, 91250 integer (91250 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
Presolve removed 185420 rows and 50 columns
Presolve time: 0.07s

Explored 0 nodes (0 simplex iterations) in 0.20 seconds (0.11 work units)
Thread count was 1 (of 18 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
Model is infeasible


In [20]:
x[0,0,0].X

NameError: name 'x' is not defined